# Deep learning Vision Technology Using YOLO

<img src="images/image_YOLO.jpg">

### 딥러닝 기반 비전 인식 기술은 제조 현장의 품질 관리와 생산성 향상에 혁신적인 솔루션을 제공할 수 있다.

- 품질 관리와 생산성 향상
    - 기존의 제조 공정에서는 사람이 직접 육안으로 제품의 결함을 검사하거나, 규격에 맞는 부품을 확인해야 했다.
    - 이 과정은 시간이 오래 걸리고, 사람의 실수로 인해 오차가 발생할 수 있으며, 반복적인 작업으로 인한 피로가 생산성을 저해하는 주요 원인이다.

- 딥러닝 비전 인식 기술은 이러한 한계를 극복할 수 있다.
    - 정확하고 일관된 검사: 딥러닝 모델은 미세한 결함이나 불량품을 사람의 눈보다 훨씬 정확(?)하고 빠르게 식별할 수 있다.
    - 24시간 무정차 가동: 사람과 달리 딥러닝 기반 시스템은 피로를 느끼지 않으므로 24시간 내내 일정한 품질로 검사를 수행할 수 있다. (음.. 이건 확실히 그렇지.)

- 복잡한 문제 해결
    - 기존의 규칙 기반 (Rule-based) 비전 시스템은 새로운 유형의 결함이나 예측 불가능한 변수에 대응하기 어렵다.
        - 딥러닝은 수많은 이미지 데이터를 학습하여 결함의 특징을 스스로 파악하고, 복잡한 패턴을 인식하는 능력을 갖췄다.
        - 제품의 미세한 변형, 비정형적인 결함 등도 파악할 수 있다. (글쎄나...)

        <img src="images/image_question.png" width="200"><br>

        - 아, 뭐, 그랬건 어쨌건 함 시작해봅시다.

In [ ]:
!pip install ultralytics
!pip install pyside6
!pip install qdarkstyle

- 비전 인식이니까 카메라부터 켜 봅시다.

In [3]:
import cv2

# 기본 카메라(일반적으로 노트북에 내장된 웹캠)에 연결합니다.
# '0'은 시스템에 연결된 첫 번째 카메라를 의미합니다.
cap = cv2.VideoCapture(0)

# 카메라가 제대로 열렸는지 확인합니다.
# 실패하면 오류 메시지를 출력하고 프로그램을 종료합니다.
if not cap.isOpened():
    print("오류: 비디오 스트림을 열 수 없습니다.")
    exit()

# 무한 루프를 시작하여 카메라에서 프레임을 계속 읽어옵니다.
while True:
    # 비디오 스트림에서 프레임(이미지)을 하나씩 읽어옵니다.
    # ret: 프레임을 성공적으로 읽었는지 여부를 나타내는 불리언 값 (True/False).
    # frame: 읽어온 실제 이미지 데이터 (넘파이 배열 형태).
    ret, frame = cap.read()

    # 프레임을 성공적으로 읽었을 경우...
    if ret:
        # "Webcam"이라는 이름의 창에 현재 프레임을 표시합니다.
        # 창 이름은 원하는 대로 변경할 수 있습니다.
        cv2.imshow("Webcam", frame)

        # 키보드 입력을 1밀리초 동안 기다립니다.
        # 이 함수는 비디오가 정상적으로 표시되고 사용자 입력에 반응할 수 있도록 해줍니다.
        key = cv2.waitKey(1)

        # 'q' 키가 눌렸는지 확인합니다.
        if key == ord('q'):
            # 'q' 키가 눌렸으면 루프를 종료합니다.
            break
    else:
        # 프레임을 읽는 데 실패하면 (예: 카메라 연결 끊김) 루프를 종료합니다.
        print("오류: 프레임을 읽을 수 없습니다.")
        break

# 사용이 끝난 카메라 객체를 해제하여 시스템 리소스를 반환합니다.
cap.release()

# 모든 OpenCV 창을 닫습니다.
cv2.destroyAllWindows()

- 카메라를 켰으니, 이제는 아예 Object Detection을 해볼까나?
    - MScoco dataset을 이용해서 미리 학습한 Object Detection 모델을 사용해보자.
    - YOLOV8n을 이용하는 방법

- YOLO란 무엇인가???
    - YOLO(You Only Look Once)는 실시간 객체 감지(Object Detection)에 특화된 딥러닝 알고리즘이다.
        - 기존의 객체 감지 모델들이 여러 단계를 거치는 것과 달리, YOLO는 이미지 전체를 한 번에 처리하여 객체의 위치와 종류를 동시에 예측한다. (You Only Look Once.. Live가 아니라 Look이다.)
    - YOLO의 작동 방식
        - YOLO는 이미지를 그리드(Grid)로 나누어 처리하는 방식을 사용.
            - 그리드 분할: 입력 이미지를 n×n 크기의 그리드로 나눈다.
        - 바운딩박스(Bounding box) 예측: 각 그리드 셀은 자신이 포함하는 객체를 확인하고, 해당 객체의 경계를 둘러싼 바운딩박스와 클래스(Class)를 계산한다.
        - 확률 점수 부여: 각 경계 상자는 두 가지 확률을 계산한다.
            - 객체 존재 확률: 해당 경계 상자에 객체가 있을 확률.
            - 클래스 확률: 객체가 특정 클래스(예: 사람, 자동차)일 확률.
        - YOLO는 이러한 과정을 통해 하나의 신경망으로 이미지 내의 모든 객체를 한 번에 감지한다.
            - 매우 빠른 속도로 객체 감지가 가능하며, 실시간 비전 인식 분야에서 널리 활용.
        - YOLO의 주요 특징
            - 빠른 속도: YOLO의 가장 큰 장점은 뛰어난 속도. 단일 신경망을 사용해 연산량을 획기적으로 줄여 실시간 애플리케이션에 적합.
            - 통합된 구조: 객체 분류와 위치 파악을 한 번에 수행하여 효율성인 높다. (같은 말임...)
            - 전역적 정보 활용: 이미지를 그리드로 나누어 처리하면서도 이미지 전체의 맥락을 고려한다. 이는 객체 간의 관계를 파악하는 데 도움을 주어 오탐지를 줄인다.
    - YOLO의 발전
        - YOLO는 초기 모델인 YOLOv1부터 시작하여, 성능과 정확도를 개선한 YOLOv2(YOLO9000), YOLOv3, YOLOv4, 그리고 최근의 YOLOv8, YOLOv11까지 다양한 버전으로 발전해왔다.
        - 각 버전은 다음과 같은 개선 사항을 포함한다.
            - 정확도 향상: 복잡한 네트워크 구조와 새로운 학습 기법을 도입하여 더 정확한 객체 감지.
            - 다양한 크기의 객체 감지: 작은 객체부터 큰 객체까지 다양한 크기의 객체를 더 잘 감지하도록 개선.
            - 효율성 증대: 모델의 경량화를 통해 모바일 기기나 임베디드 시스템에서도 구동이 가능하도록.

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if ret:
        results = model(frame, verbose=False)
        frame = results[0].plot()
        cv2.imshow("yolov8", frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

WARNING Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\greathtj\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


- 그러니까 여기에 우리의 사물(생산현장에서 발견해야하는 부품 및 이상, 혹은 양품/비품)을 인식하도록 하면 되는 거다!!!

- Object Detection 과정

1. 데이터 수집 및 준비
    - 사진 및 영상 촬영: 다양한 환경, 조명, 각도에서 객체가 포함된 사진이나 영상을 직접 촬영한다.
    - 데이터 증강: 촬영된 데이터를 기반으로 회전, 반전, 밝기 조절 등을 통해 데이터셋의 양을 늘린다.

2. 데이터 라벨링 (Annotation)
    - 수동 라벨링: 이미지 속의 객체 주변에 경계 상자(Bounding Box)를 그리고, 객체 종류(ex: 사람, 자동차, 고양이)를 수동으로 지정한다.
    - 라벨링 포맷: YOLO, COCO, PASCAL VOC 등 널리 사용되는 형식으로 라벨 파일을 저장한다.

3. 모델 학습 (Training)
    - 모델 선택: YOLO, Faster R-CNN, SSD 등 목적에 맞는 딥러닝 모델을 선택한다.
    - 데이터 분할: 전체 데이터셋을 학습(Training), 검증(Validation), 테스트(Test) 세트로 나눈다.
    - 학습: 라벨링된 데이터를 이용하여 모델을 학습시킨다. 
        - 모델은 반복적인 학습을 통해 객체의 특징을 인식하고 정확도를 높인다.

4. 모델 성능 평가
    - 정확도 측정: 모델이 얼마나 정확하게 객체를 탐지하는지 정밀도(Precision), 재현율(Recall), mAP(mean Average Precision) 등의 지표를 사용해 평가한다.
    - 오차 분석: 모델이 잘못 탐지한 사례(오탐지)나 놓친 사례(미탐지)를 분석하여 개선점을 찾는다.

5. 모델 배포 (Deployment)
    - 최적화: 모바일, 웹, 에지 디바이스 등 실제 환경에 맞게 모델의 크기와 속도를 최적화한다.
    - 통합: 학습된 모델을 제조공정에 통합한다. (ex: 전용 비전인식장치, 제조현장 CCTV 시스템, 자율제조시스템)

6. 피드백 및 개선
    - 성능 모니터링: 실제 환경에서 모델의 성능을 지속적으로 모니터링한다.
    - 피드백 루프: 사용자 피드백이나 실제 환경에서 발생한 오류 데이터를 수집하여 모델 학습 데이터셋에 추가한다.  
        - 이 과정을 반복하여 모델의 성능을 지속적으로 향상시킨다. (늘 그렇지만 이것도 노가다중에 노가다다.)

        <img src="images/image_working.png" width="200"><br>

- 떠라나, 노가를 성공적으로 마친 당신

In [1]:
import cv2
from ultralytics import YOLO

model = YOLO("best.pt")
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if ret:
        results = model(frame, verbose=False)
        frame = results[0].plot()
        cv2.imshow("yolov8", frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

- 어마나!! 이거 정말 되네??!!

- 그럼 이걸 제조 설비와 연동하게 할 수 있나?
    - 당연히 할 수 있지... (여러분들은 이미 MCU를 제어할 수 있지 않은가!!)

        https://www.youtube.com/watch?v=xJivazy4rRE

        https://www.youtube.com/watch?v=h_SokA6XDeE
